# Dynamic Salary Retirement Model

Table of content:
- [**Setup**](#Setup): Runs any imports or other setup
- [**Inputs**](#Inputs): Defines the input of the model
- [**Salaries**](#Salaries): Determines the annual salary based on cost of living and promotion raises.
- [**Wealth**](#Wealth): Calculates the wealth generating over the years.
- [**Retirement**](#Retirement): Calculates the Retirement Corpus after each year.

# Setup

Setups for the later calculations are here. The necessary packages are imported.

In [22]:
from dataclasses import dataclass
import pandas as pd

# Inputs 
All the inputs for the model are defined here. A class is contructed to manage the data, and an instance of the class containing the default inputs is created.

In [3]:
@dataclass
class ModelInputs:
    starting_salary: int = 60000
    promos_after_every_n_years: int = 5
    cost_of_living_raise: float = 0.02
    promo_raise: float = 0.15
    savings_rate: float = 0.25
    interest_rate: float = 0.05
    desired_cash: int = 1500000

model_data = ModelInputs()
model_data

ModelInputs(starting_salary=60000, promos_after_every_n_years=5, cost_of_living_raise=0.02, promo_raise=0.15, savings_rate=0.25, interest_rate=0.05, desired_cash=1500000)

In [4]:
data = model_data

## Salaries

Using the Formula : $S_t = S_0(1 + r_l)^t (1 + r_p)^p$ we will calculate the salaries at different years.<br><br>
where
- $S_t$ : Salary at year $t$
- $S_0$ : Starting Salary
- $r_l$ : Return for cost of living
- $r_p$ : Return for Promotion
- $t$ : Number of Years
- $p$ : Number of Promotions

In [5]:
def salary_at_year(data, year):
    """
    Using this function will give you the salary at that particular year.
    """
    num_promos = int(year/data.promos_after_every_n_years)

    salary_t = data.starting_salary * (1 + data.cost_of_living_raise)**year * (1 + data.promo_raise)** num_promos
    return salary_t

In [43]:
for i in range(6):
    year = i +1
    salary = salary_at_year(model_data, year)
    print(f'The Salary at year {year} is ${salary:,.0f}')

The Salary at year 1 is $61,200
The Salary at year 2 is $62,424
The Salary at year 3 is $63,672
The Salary at year 4 is $64,946
The Salary at year 5 is $76,182
The Salary at year 6 is $77,705


## Wealth

Using the formula : $W_t = W_t-1 (1 + r_i) + S_t v$ we will calculate the Wealth generated at different years. <br><br>where
- $S_t$ : Salary at year $t$
- $W_t$ : Wealth at year $t$
- $r_i$ : Investment Return
- $t$ : Number of years
- $v$ : Savings Rate

In [44]:
def cash_saved_during_year(data, year):
    salary = salary_at_year(data, year)
    cash_saved = salary * data.savings_rate
    return cash_saved
cash_saved_during_year(model_data, 15)

30703.46263573066

In [45]:
def wealth_at_year(data, year, prior_wealth):
    cash_saved = cash_saved_during_year(data, year)
    wealth = prior_wealth * (1 + data.interest_rate) + cash_saved
    return wealth

In [46]:
prior_wealth = 0
for i in range(6):
    year = i + 1
    wealth = wealth_at_year(model_data, year, prior_wealth)
    print(f'The wealth at year {year} is ${wealth:,.0f}')

    prior_wealth = wealth

The wealth at year 1 is $15,300
The wealth at year 2 is $31,671
The wealth at year 3 is $49,173
The wealth at year 4 is $67,868
The wealth at year 5 is $90,307
The wealth at year 6 is $114,248


## Retirement

In [47]:
def years_to_retirement(data):

    # starting with no cash saved
    prior_wealth = 0
    wealth = 0

    year = 0 # will become 1 on first loop

    print('Wealth over time:') # \n makes a blank line in the outpu
    while wealth < data.desired_cash:
        year = year + 1
        wealth = wealth_at_year(data, year, prior_wealth)
        print(f'The wealth at year {year} is ${wealth:,.0f}')

        # set next year's prior wealth to this year's wealth
        prior_wealth = wealth
    
    # Now we have exited the loop, so wealth must be >= desired_cash. Whatever last year was set
    # is the years to retirement   
    print(f'\nRetirement:\nIt will take you {year} years to retire.')
    return year

In [48]:
ytr = years_to_retirement(model_data)

# Results Summary

In [34]:
def get_salaries_wealths_df(data):
    # Starting with no cash saved
    prior_wealth = 0
    wealth = 0
        
    year = 0 # will become 1 on first loop

    df_data_tups = []
    while wealth < data.desired_cash:
        year = year + 1
        salary = salary_at_year(data, year)
        wealth = wealth_at_year(model_data, year, prior_wealth)

        # set next year's prior wealth to this year's wealth
        prior_wealth = wealth

        df_data_tups.append((year, salary, wealth))

    df = pd.DataFrame(df_data_tups, columns = ['Year', 'Salary', 'Wealth'])
    return df

In [35]:
df = get_salaries_wealths_df(model_data)
df

,Year,Salary,Wealth
0,1,61200.000000,1.530000e+04
1,2,62424.000000,3.167100e+04
2,3,63672.480000,4.917267e+04
3,4,64945.929600,6.786779e+04
4,5,76181.575421,9.030657e+04
5,6,77705.206929,1.142482e+05
6,7,79259.311068,1.397754e+05
7,8,80844.497289,1.669753e+05
8,9,82461.387235,1.959394e+05
9,10,96727.207227,2.299182e+05


In [ ]:
df.style.format({
    'Salary': '${:,.0f}',
    'Wealth': '${:,.0f}',
}).bar(subset=['Salary', 'Wealth'])